## Data Mining Project - Deliverable 1

In [ ]:
# import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import openpyxl 

In [10]:

df_customerDB = pd.read_csv('DM_AIAI_CustomerDB.csv')
df_flightsDB = pd.read_csv('DM_AIAI_FlightsDB.csv')
df_metadata = pd.read_csv('DM_AIAI_Metadata.csv', sep=';') # this file is not comma separated but semicolon separated

In [7]:
#DESCRIPTIVE STATISTICS AND VISUALIZATIONS
# ver head(), info(), describe( of each dataframe)
# fazer histogramas e boxplots das variáveis 

#ASSESS DATA QUALITY ISSUES AND CLUSTERING RELIABILITY ??
# check for missing values, duplicates, inconsistencies, outliers

#IDENTIFY PRELIMINARY BEHAVIORAL SIGNALS
# ver correlações e scatter plots entre as features

#DEVELOP AND JUSTIFY ENGINEERED FEATURES

In [12]:
display(df_metadata)

,CustomerDB,Unnamed: 1,Unnamed: 2,Unnamed: 3
0,NaN,Variable,Description,NaN
1,NaN,Loyalty#,Unique customer identifier for loyalty program...,NaN
2,NaN,First Name,Customer's first name,NaN
3,NaN,Last Name,Customer's last name,NaN
4,NaN,Customer Name,Customer's full name (concatenated),NaN
5,NaN,Country,Customer's country of residence,NaN
6,NaN,Province or State,Customer's province or state,NaN
7,NaN,City,Customer's city of residence,NaN
8,NaN,Latitude,Geographic latitude coordinate of customer loc...,NaN
9,NaN,Longitude,Geographic longitude coordinate of customer lo...,NaN


In [19]:
metadata_customerDB = df_metadata.loc[1:20,['Unnamed: 1', 'Unnamed: 2']]
metadata_customerDB.columns = ['Variable', 'Description']
display(metadata_customerDB)

,Variable,Description
1,Loyalty#,Unique customer identifier for loyalty program...
2,First Name,Customer's first name
3,Last Name,Customer's last name
4,Customer Name,Customer's full name (concatenated)
5,Country,Customer's country of residence
6,Province or State,Customer's province or state
7,City,Customer's city of residence
8,Latitude,Geographic latitude coordinate of customer loc...
9,Longitude,Geographic longitude coordinate of customer lo...
10,Postal code,Customer's postal/ZIP code
